In [1]:
using StatisticalRethinking, CmdStan
gr(size=(500,500));

ProjDir = rel_path("..", "scripts", "04")
cd(ProjDir)

howell1 = CSV.read(rel_path("..", "data", "Howell1.csv"), delim=';')
df = convert(DataFrame, howell1);

df2 = filter(row -> row[:age] >= 18, df)
first(df2, 5)

heightsmodel = "
// Inferring a Rate
data {
  int N;
  real<lower=0> h[N];
}
parameters {
  real<lower=0> sigma;
  real<lower=0,upper=250> mu;
}
model {
  // Priors for mu and sigma
  mu ~ normal(178, 20);
  sigma ~ uniform( 0 , 50 );

  // Observed heights
  h ~ normal(mu, sigma);
}
";

stanmodel = Stanmodel(name="heights", monitors = ["mu", "sigma"],model=heightsmodel,
  output_format=:mcmcchains);

heightsdata = Dict("N" => length(df2[:height]), "h" => df2[:height]);

rc, chn, cnames = stan(stanmodel, heightsdata, ProjDir, diagnostics=false,
  CmdStanDir=CMDSTAN_HOME);

describe(chn)

serialize("m4.1s.jls", chn)
chn2 = deserialize("m4.1s.jls")

describe(chn2)


File /var/folders/fb/m164mwfn3v1btnrpmmyzpvqh0000gn/T/jl_z36fME/heights.stan will be updated.


Inference for Stan model: heights_model
4 chains: each with iter=(1000,1000,1000,1000); warmup=(0,0,0,0); thin=(1,1,1,1); 4000 iterations saved.

Warmup took (0.033, 0.031, 0.033, 0.033) seconds, 0.13 seconds total
Sampling took (0.063, 0.060, 0.040, 0.043) seconds, 0.21 seconds total

                Mean     MCSE  StdDev    5%   50%   95%    N_Eff  N_Eff/s    R_hat
lp__            -892  2.3e-02    0.98  -893  -891  -891  1.8e+03  9.0e+03  1.0e+00
accept_stat__   0.92  1.5e-03   0.099  0.72  0.96   1.0  4.5e+03  2.2e+04  1.0e+00
stepsize__      0.79      nan   0.084  0.70  0.78  0.93      nan      nan  5.1e+13
treedepth__      2.1  9.3e-02    0.77   1.0   2.0   3.0  6.7e+01  3.3e+02  1.0e+00
n_leapfrog__     5.2  7.5e-01     6.2   1.0   3.0    15  6.9e+01  3.4e+02  1.0e+00
divergent__     0.00      nan    0.00  0.00  0.00  0.00      nan      nan      nan
energy__         892  3.4e-02     1

2-element Array{ChainDataFrame,1}

Summary Statistics
. Omitted printing of 1 columns
│ Row │ parameters │ mean    │ std      │ naive_se   │ mcse       │ ess     │
│     │ Symbol     │ Float64 │ Float64  │ Float64    │ Float64    │ Any     │
├─────┼────────────┼─────────┼──────────┼────────────┼────────────┼─────────┤
│ 1   │ mu         │ 154.613 │ 0.415611 │ 0.00657139 │ 0.00855796 │ 3360.01 │
│ 2   │ sigma      │ 7.76704 │ 0.293409 │ 0.0046392  │ 0.0049645  │ 3018.16 │

Quantiles

│ Row │ parameters │ 2.5%    │ 25.0%   │ 50.0%   │ 75.0%   │ 97.5%   │
│     │ Symbol     │ Float64 │ Float64 │ Float64 │ Float64 │ Float64 │
├─────┼────────────┼─────────┼─────────┼─────────┼─────────┼─────────┤
│ 1   │ mu         │ 153.82  │ 154.336 │ 154.614 │ 154.893 │ 155.422 │
│ 2   │ sigma      │ 7.2095  │ 7.56297 │ 7.76808 │ 7.95756 │ 8.37963 │


end of m4.1s#-
*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*